In [1]:
import numpy as np
# import cv2
# import trimesh
# from torch import sigmoid
# from vgn.ConvONets.utils import libmcubes
# import mcubes

In [ ]:
import os
len(os.listdir('data/pile/data_pile_train_raw_4M/scenes'))

In [9]:
di = {"Train" + str(k):v for k, v in enumerate(np.random.rand(5))}

In [11]:
msg = 'Val'
for k, v in di.items():
    msg += f' {k}: {v:.4f}'

print(msg)

Val Train0: 0.3900 Train1: 0.5451 Train2: 0.6462 Train3: 0.3115 Train4: 0.4339


# Generate dataset log file

In [5]:
import pandas as pd
from pathlib import Path

dataset = Path('data_pile_train_random_raw_4M')
df = pd.read_csv('data/pile/data_pile_train_random_raw_4M/grasps.csv')
len(df)

623934

In [6]:
counts_df = df.groupby('scene_id').size().reset_index(name='count')
labels_df = df.groupby(['scene_id', 'label']).size().reset_index(name='count')
filename = 'summary_%s.txt' % (dataset.name)
with open(filename, 'w') as f:
    summary = "Dataset description: \n Total samples: %d\n Total number of scenes: %d \n Max samples of scenes: %d \n Mean samples of scenes: %d \n Min samples of scenes: %d \n\n" % (len(df), len(counts_df), counts_df['count'].max(), int(counts_df['count'].mean()), counts_df['count'].min())
    f.write(summary)

counts_df.to_csv(filename, mode = 'a', sep='\t', header=True, index=False)
labels_df.to_csv(filename, mode = 'a', sep='\t', header=True, index=False)

In [7]:
with open(filename, 'r') as f:
    print((";").join(f.readlines()[1:5]).replace('\n', ''))

 Total samples: 623934; Total number of scenes: 33311 ; Max samples of scenes: 74 ; Mean samples of scenes: 18 


# Test

In [ ]:
input_sdf = np.load('input_sdf.npy')
recon_sdf = np.load('recon_sdf.npy')
#recon_sdf = sigmoid(torch.tensor(recon_sdf)).numpy().reshape(20, 20, 20)

In [112]:
len(np.where(input_sdf>=0.5)[0])

3132

In [88]:
input_sdf.min(), input_sdf.max(), recon_sdf.min(), recon_sdf.max()

(0.0, 0.98990965, -18.649174, 1.9090331)

In [89]:
box_size = 1 + 0.1
threshold = 0
n_x, n_y, n_z = recon_sdf.shape
# occ_hat_padded = np.pad(
#             recon_sdf, 1, 'constant', constant_values=-1e6)
vertices, triangles = mcubes.marching_cubes(
            recon_sdf, threshold)

In [90]:
vertices -= 0.5
vertices /= np.array([n_x-1, n_y-1, n_z-1])
vertices = box_size * (vertices - 0.5)

In [8]:
# recon_sdf

In [1]:
import trimesh
mesh = trimesh.load('/home/hypatia/6D-DAAD/unisurf/out/giga/scene001/extraction/meshes/scan_world_scale.obj')

In [2]:
mesh

<trimesh.Trimesh(vertices.shape=(2798691, 3), faces.shape=(5601248, 3))>

In [2]:
mesh.show()

In [4]:
from vgn.utils.implicit import get_scene_from_mesh_pose_list
import numpy as np

mesh_pose_list  = np.load('unisurf_data_pc/scene_1.npz', allow_pickle=True)['pc']
mesh = get_scene_from_mesh_pose_list(mesh_pose_list)
print(mesh)
mesh.show()

['pc']